In [1]:
import sys
sys.path.append('..')
from tool import *

In [1]:
# Organize data
def get_stat(sza_L_mode,band_mode,landcover_type):
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15  
    _,sza_L=torch.load(root_path+'sza_and_szaL.pth')
    if band_mode=='shortwave':  
        WB_sky_fraction=torch.load("/data/hk/albedo/white_sky_fraction/white_sky_fraction2.pth")
    else:
        if band_mode not in ['nir','vis']:
            raise('Error band_mode!')
        WB_sky_fraction=torch.load(f"/data/hk/albedo/white_sky_fraction/white_sky_fraction_{band_mode}2.pth")
    landtypes=['landtype'+str(i) for i in range(1,lc_len+1)]+['snow']
    stat={}
    for month in range(1,13):
        NC=nc.Dataset(f"/data2/hzy/albedo2/albedo_information_nc2/fill/month_{month}_{landcover_type}.nc",'r')
        for year in range(2001,2021):
            types_stat_albedo=[]
            types_stat_area=[]
            for type in landtypes:
                bs=NC.variables[f'albedo_BSA_{band_mode}-{type}'][year-2001]
                ws=NC.variables[f'albedo_WSA_{band_mode}-{type}'][year-2001]
                area_type=NC.variables[f'area-{type}'][year-2001] 
                area_land=NC.variables[f'area-land'][year-2001] 
                wf=WB_sky_fraction[(year-2001)*12+month-1]
                if sza_L_mode=='70':
                    area_type[sza_L[year-2001,month-1]>=1]=np.nan###
                elif sza_L_mode=='85':
                    area_type[sza_L[year-2001,month-1]>=2]=np.nan###
                else:
                    raise('Error sza_L_mode')
                types_stat_albedo.append(((ws*wf+bs*(1-wf))/1000))      
                types_stat_area.append(area_type)
            types_stat_albedo=np.stack(types_stat_albedo)
            types_stat_area=np.stack(types_stat_area)
            types_stat_albedo[types_stat_albedo==0]=np.nan
            stat[f'{year}-{month}-albedo']=types_stat_albedo
            stat[f'{year}-{month}-typearea']=types_stat_area
            stat[f'{year}-landarea']=area_land   
        NC.close() 
    save_path=f'/data2/hzy/albedo2/data/stat_sw-mean_sza-{sza_L_mode}_band-{band_mode}_{landcover_type}.pth'
    print(save_path)
    torch.save(stat,save_path)###

In [2]:
# Organize data
for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
    for sza_L_mode in ['70','85']:
        for band_mode in ['shortwave']:
            get_stat(sza_L_mode,band_mode,landcover_type)

/data2/hzy/albedo2/data/stat_sw-mean_sza-70_band-shortwave_LCCS1.pth
/data2/hzy/albedo2/data/stat_sw-mean_sza-85_band-shortwave_LCCS1.pth
/data2/hzy/albedo2/data/stat_sw-mean_sza-70_band-shortwave_LCCS2.pth
/data2/hzy/albedo2/data/stat_sw-mean_sza-85_band-shortwave_LCCS2.pth
/data2/hzy/albedo2/data/stat_sw-mean_sza-70_band-shortwave_LCCS3.pth
/data2/hzy/albedo2/data/stat_sw-mean_sza-85_band-shortwave_LCCS3.pth
/data2/hzy/albedo2/data/stat_sw-mean_sza-70_band-shortwave_IGBP.pth
/data2/hzy/albedo2/data/stat_sw-mean_sza-85_band-shortwave_IGBP.pth


In [ ]:
#Calculate the albedo change curve and seasonal variance.
def get_01_season(s,e,landcover_type,mode='region',snowfree=True):
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15 
    kk=[[],[],[],[]]
    data_m=torch.load(f"/data2/hzy/albedo2/confuse_matrix/confuse_matrix_2020_{landcover_type}.pth")
    mask_t=np.expand_dims(mask[:,0,:],0).repeat(lc_len+1,axis=0).transpose((1,3,4,0,2))
    data_m[mask_t>=1]=np.nan
    data_m=month_fill(data_m,mask_t>=1)
    if snowfree:
        data_m[:,:,:,-1,:]=0
        data_m[:,:,:,:,-1]=0
    z=albedo_all[:,0].transpose((0,2,3,1)).reshape((12,180,360,lc_len+1,1))*data_m
    n=0
    for month in [[3,4,5],[6,7,8],[9,10,11],[12,1,2]]:
        if mode=='all':
            tt=[z[i-1,:] for i in month]
        else:
            tt=[z[i-1,s:e] for i in month]
        mts=np.array([mt[i-1] for i in month]).reshape(3,1,1,1,1)
        tt=np.stack(tt)*mts/np.sum(mts)
        if mode=='all':
            mm_area=np.array([np.nansum(data_m[i-1,:]) for i in month]).mean()
        else:
            mm_area=np.array([np.nansum(data_m[i-1,s:e]) for i in month]).mean()
        kk[n].append(np.nansum(tt)/mm_area)
        n+=1
    return kk
def get_01_all(s,e,landcover_type,mode='region',snowfree=True):
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15 
    data_m=torch.load(f"/data2/hzy/albedo2/confuse_matrix/confuse_matrix_2020_{landcover_type}.pth")
    mask_t=np.expand_dims(mask[:,0,:],0).repeat(lc_len+1,axis=0).transpose((1,3,4,0,2))
    data_m[mask_t>=1]=np.nan
    data_m=month_fill(data_m,mask_t>=1)
    if snowfree:
        data_m[:,:,:,-1,:]=0
        data_m[:,:,:,:,-1]=0
    z=albedo_all[:,0].transpose((0,2,3,1)).reshape((12,180,360,lc_len+1,1))*data_m
    z[np.isnan(z)]=0
    z=z*(mt.reshape(12,1,1,1,1)/365/24/3600)
    if mode=='all':
        return [np.nansum(z[:,:])/np.nansum(np.nanmean(data_m[:,:],axis=0))]
    else:
        return [np.nansum(z[:,s:e])/np.nansum(np.nanmean(data_m[:,s:e],axis=0))]
def get_curve(s,t,snowfree,ismask=False):
    season=[]
    kk=get_01_season(s,t,landcover_type,snowfree=snowfree)
    for y in range(2002,2021):
        data_m=torch.load(f"/data2/hzy/albedo2/confuse_matrix/confuse_matrix_{y}_{landcover_type}.pth")
        mask_t=np.expand_dims(mask[:,0,:],0).repeat(lc_len+1,axis=0).transpose((1,3,4,0,2))
        data_m[mask_t>=1]=np.nan
        data_m=month_fill(data_m,mask_t>=1)
        if snowfree:
            data_m[:,:,:,-1,:]=0
            data_m[:,:,:,:,-1]=0
        z=albedo_all[:,y-2001].transpose((0,2,3,1)).reshape((12,180,360,1,lc_len+1))*data_m
        n=0
        for month in [[3,4,5],[6,7,8],[9,10,11],[12,1,2]]:
            tt=[z[i-1,s:t] for i in month]
            mts=np.array([mt[i-1] for i in month]).reshape(3,1,1,1,1)
            tt=np.stack(tt)*mts/np.sum(mts)
            mm_area=np.array([np.nansum(data_m[i-1,s:t]) for i in month]).mean()
            kk[n].append(np.nansum(tt)/mm_area)
            n+=1
    for i in range(4):
        kk2=[0]+[kk[i][j]-kk[i][0] for j in range(1,20)]
        season.append(kk2)
    stds=[np.std([season[0][i],season[1][i],season[2][i],season[3][i]]) for i in range(20)]
    maxs=[np.max([season[0][i],season[1][i],season[2][i],season[3][i]]) for i in range(20)]
    mins=[np.min([season[0][i],season[1][i],season[2][i],season[3][i]]) for i in range(20)]
    kk2=get_01_all(s,t,landcover_type,snowfree=snowfree)
    for y in tqdm(range(2002,2021)):
        data_m=torch.load(f"/data2/hzy/albedo2/confuse_matrix/confuse_matrix_{y}_{landcover_type}.pth")
        mask_t=np.expand_dims(mask[:,0,:],0).repeat(lc_len+1,axis=0).transpose((1,3,4,0,2))
        data_m[mask_t>=1]=np.nan
        data_m=month_fill(data_m,mask_t>=1)
        if snowfree:
            data_m[:,:,:,-1,:]=0
            data_m[:,:,:,:,-1]=0
        z=albedo_all[:,y-2001].transpose((0,2,3,1)).reshape((12,180,360,1,lc_len+1))*data_m
        z[np.isnan(z)]=0
        z=z*(mt.reshape(12,1,1,1,1)/365/24/3600)
        kk2.append(np.nansum(z[:,s:t])/np.nansum(np.nanmean(data_m[:,s:t],axis=0)))
    kk=[0]+[kk2[j]-kk2[0] for j in range(1,len(kk2))]
    return kk2,stds,maxs,mins

In [ ]:
#Calculate the albedo change curve and seasonal variance.
mt=[]
for month in range(1,13):
    now=datetime.datetime.strptime(f'2001-{month}-1', '%Y-%m-%d')
    month_time=(now+relativedelta(months=1)-now).total_seconds()
    mt.append(month_time)
mt=np.array(mt)
data=torch.load('../data/curve.pth')
for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
    print(landcover_type)
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15 
    stat=torch.load(f'/data2/hzy/albedo2/data/stat_sw-mean_sza-70_band-shortwave_{landcover_type}.pth')
    _,sza_L=torch.load(root_path+'sza_and_szaL.pth')
    albedo_all=np.zeros((12,20,lc_len+1,180,360)).astype(np.float32)
    area_all=np.zeros((12,20,lc_len+1,180,360)).astype(np.float32)
    mask=np.zeros((12,20,lc_len+1,180,360))

    for y in range(2001,2021):
        for m in range(1,13):
            albedo_all[m-1,y-2001]=stat[f'{y}-{m}-albedo']
            area_all[m-1,y-2001]=stat[f'{y}-{m}-typearea']
            mask[m-1,y-2001]=np.expand_dims(sza_L[y-2001,m-1],0).repeat(lc_len+1,axis=0)

    albedo_all[albedo_all==0]=np.nan
    albedo_all[mask>=1]=np.nan
    area_all=month_fill(area_all,mask>=1)
    albedo_all=month_fill(albedo_all,mask>=1)  


    kk2,stds,maxs,mins=get_curve(0,180,False)
    data[f'G_snow_{landcover_type}']=kk2
    data[f'G_snow_{landcover_type}_std']=stds
    data[f'G_snow_{landcover_type}_max']=maxs
    data[f'G_snow_{landcover_type}_min']=mins
    
    kk2,stds,maxs,mins=get_curve(0,180,True)
    data[f'G_snowfree_{landcover_type}']=kk2
    data[f'G_snowfree_{landcover_type}_std']=stds
    data[f'G_snowfree_{landcover_type}_max']=maxs
    data[f'G_snowfree_{landcover_type}_min']=mins

    kk2,stds,maxs,mins=get_curve(0,24,False)
    data[f'N_snow_{landcover_type}']=kk2
    data[f'N_snow_{landcover_type}_std']=stds
    data[f'N_snow_{landcover_type}_max']=maxs
    data[f'N_snow_{landcover_type}_min']=mins
    
    kk2,stds,maxs,mins=get_curve(-24,180,False)
    data[f'S_snow_{landcover_type}']=kk2
    data[f'S_snow_{landcover_type}_std']=stds
    data[f'S_snow_{landcover_type}_max']=maxs
    data[f'S_snow_{landcover_type}_min']=mins
    
    kk2,stds,maxs,mins=get_curve(24,90-23,False)
    data[f'TN_snow_{landcover_type}']=kk2
    data[f'TN_snow_{landcover_type}_std']=stds
    data[f'TN_snow_{landcover_type}_max']=maxs
    data[f'TN_snow_{landcover_type}_min']=mins
    
    kk2,stds,maxs,mins=get_curve(24,90-23,True)
    data[f'TN_snowfree_{landcover_type}']=kk2
    data[f'TN_snowfree_{landcover_type}_std']=stds
    data[f'TN_snowfree_{landcover_type}_max']=maxs
    data[f'TN_snowfree_{landcover_type}_min']=mins
    
    kk2,stds,maxs,mins=get_curve(90-23,90+24,False)
    data[f'T_snow_{landcover_type}']=kk2
    data[f'T_snow_{landcover_type}_std']=stds
    data[f'T_snow_{landcover_type}_max']=maxs
    data[f'T_snow_{landcover_type}_min']=mins
    
    kk2,stds,maxs,mins=get_curve(90-23,90+24,True)
    data[f'T_snowfree_{landcover_type}']=kk2
    data[f'T_snowfree_{landcover_type}_std']=stds
    data[f'T_snowfree_{landcover_type}_max']=maxs
    data[f'T_snowfree_{landcover_type}_min']=mins
    
    kk2,stds,maxs,mins=get_curve(90+24,-24,False)
    data[f'TS_snow_{landcover_type}']=kk2
    data[f'TS_snow_{landcover_type}_std']=stds
    data[f'TS_snow_{landcover_type}_max']=maxs
    data[f'TS_snow_{landcover_type}_min']=mins
    
    kk2,stds,maxs,mins=get_curve(90+24,-24,True)
    data[f'TS_snowfree_{landcover_type}']=kk2
    data[f'TS_snowfree_{landcover_type}_std']=stds
    data[f'TS_snowfree_{landcover_type}_max']=maxs
    data[f'TS_snowfree_{landcover_type}_min']=mins


torch.save(data,'../data/curve.pth')

In [ ]:
# Calculate albedo-multiply-area-multiply-time
def get_aat(landcover_type):
    print(landcover_type)
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15 
    stat=torch.load(f'/data2/hzy/albedo2/data/stat_sw-mean_sza-85_band-shortwave_{landcover_type}.pth')
    _,sza_L=torch.load(root_path+'sza_and_szaL.pth')
    bluealbedo_all=np.zeros((12,20,lc_len+1,180,360)).astype(np.float32)
    area_all=np.zeros((12,20,lc_len+1,180,360)).astype(np.float32)
    mask=np.zeros((12,20,lc_len+1,180,360))
    for y in range(2001,2021):
        for m in range(1,13):
            bluealbedo_all[m-1,y-2001]=stat[f'{y}-{m}-albedo']
            area_all[m-1,y-2001]=stat[f'{y}-{m}-typearea']
            mask[m-1,y-2001]=np.expand_dims(sza_L[y-2001,m-1],0).repeat(lc_len+1,axis=0)

    bluealbedo_all[np.isnan(bluealbedo_all)]=0
    bluealbedo_all[mask>=2]=0
    area_all[np.isnan(area_all)]=0
    area_all[mask>=2]=0
    kka=[]
    kka.append(np.zeros((12,180, 360, lc_len+1, lc_len+1)))
    for y in tqdm(range(2002,2021)):
        data_m=torch.load(f"/data2/hzy/albedo2/confuse_matrix/confuse_matrix_{y}_{landcover_type}.pth")
        mask_t=np.expand_dims(mask[:,0,:],0).repeat(lc_len+1,axis=0).transpose((1,3,4,0,2))
        data_m[mask_t>=2]=np.nan
        data_m=month_fill(data_m,mask_t>=2)
        data_y=np.nanmean(data_m,axis=0)
        transf=np.nansum(data_y,axis=0)
        transf=np.nansum(transf,axis=0)
        dif_al=[]
        for m in range(12):
            exp1_2020=np.expand_dims(bluealbedo_all[m,y-2001],0).repeat(lc_len+1,axis=0)
            exp0_2001=np.expand_dims(bluealbedo_all[m,0],1).repeat(lc_len+1,axis=1)
            dif_al_m=(exp1_2020-exp0_2001).transpose((2,3,0,1))*data_m[m]
            dif_al.append(dif_al_m)
        dif_al=np.stack(dif_al)
        kka.append(dif_al)
    kka=np.stack(kka)
    kka=kka.transpose((0,4,5,1,2,3))
    kka2=kka*(mt.reshape((1,1,1,12,1,1)))
    np.save(f'/data2/hzy/albedo2/data/confusematrix_albedo-multiply-area-multiply-time_{landcover_type}.npy',kka2)
    
for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
    get_aat(landcover_type)

In [ ]:
# Calculate the RF Matrix
result2={}
for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
    print(landcover_type)
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15 
    kka=np.load(f'/data2/hzy/albedo2/data/confusematrix_albedo-multiply-area-multiply-time_{landcover_type}.npy')
    kka=kka[-1]
    kernels=torch.load(f"/data2/hzy/albedo2/kernel/kernels.pth")
    rr=[]
    for k in tqdm(['HadGEM2', 'HadGEM3', 'CAM5', 'ECHAM6', 'ERAI', 'ERA5']):
        kka2=kka*kernels[k]
        rr.append(np.nansum(kka2)/365/24/3600/5.1e8)
        result2[f'{landcover_type}_{k}']=np.nansum(kka2.reshape((lc_len+1,lc_len+1,-1)),axis=-1)/365/24/3600/5.1e8
torch.save(result2,f'../data/RF_ConfusionMatrix_2020.pth')

In [ ]:
#Calculate the RF curve for six different kernels.
for is_snowfree in ['snowdynamic','snowfree','snow']:
    kernels=torch.load(f"/data2/hzy/albedo2/kernel/kernels.pth")
    result={}
    for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
        kka=np.load(f'/data2/hzy/albedo2/data/confusematrix_albedo-multiply-area-multiply-time_{landcover_type}.npy')
        if is_snowfree=='snowdynamic':
            kka[:,:-1,:-1]=0
        elif is_snowfree=='snowfree':
            kka[:,-1,:]=0
            kka[:,:,-1]=0
        elif is_snowfree=='snow':
            pass
        else:
            raise()
        for key in ['HadGEM2', 'HadGEM3', 'CAM5', 'ECHAM6', 'ERAI', 'ERA5']:
            print(f'{landcover_type}_{key}')
            kka2=kka*kernels[key]*1e6
            kka2[np.isnan(kka)]=0
            kk2=[]
            for y in range(2001,2021):
                ay=np.nansum(kka2[y-2001,:,:])
                kk2.append(ay)
            kk=[0]+[kk2[j]-kk2[0] for j in range(1,len(kk2))]
            result[f'{landcover_type}_{key}']=kk
            # print('integrated RF',np.array(kk).sum())
            del kka2
            gc.collect()
            torch.save(result,f'curve_energy_{is_snowfree}.pth')

In [ ]:
#Export the albedo change map induce different factor change.
def filter_mode(data,mode,lc_len):
    if mode=='snowfree':
        data[:,:,:,-1,:]=0
        data[:,:,:,:,-1]=0
    elif mode=='snowfree-change':
        data[:,:,:,-1,:]=0
        data[:,:,:,:,-1]=0
    elif mode=='all':
        pass
    elif mode=='snow':
        data[:,:,:,:-1,:-1]=0
    elif mode=='conversion':
        data[:,:,:,-1,:]=0
        data[:,:,:,:,-1]=0
        for i in range(lc_len+1):
            data[:,:,:,i,i]=0       
    elif mode=='non_conversion':
        mask_filter=np.zeros((lc_len+1,lc_len+1))
        for i in range(lc_len):
            mask_filter[i,i]=1
        data=data*(mask_filter.reshape((1,1,1,lc_len+1,lc_len+1)))
    else:
        raise('ERROR MODE')
    return data
def get_albedo_change(landcover_type,mode,ismask):
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15 
    kk=[0]
    kk_map=[]
    kk_map.append(np.zeros((180,360)))
    for y in range(2002,2021):
        data_m=torch.load(f"/data2/hzy/albedo2/confuse_matrix/confuse_matrix_{y}_{landcover_type}.pth")
        mask_t=np.expand_dims(mask[:,0,:],0).repeat(lc_len+1,axis=0).transpose((1,3,4,0,2))
        data_m[mask_t>=1]=np.nan
        data_m=month_fill(data_m,mask_t>=1)
        data_m2=data_m.copy()
        data_m=filter_mode(data_m,mode,lc_len)
        dif_al=[]
        calibration=(y-2001)/10*0.007
        for m in range(12):
            exp1_2020=np.expand_dims(albedo_all[m,y-2001],0).repeat(lc_len+1,axis=0)
            exp0_2001=np.expand_dims(albedo_all[m,0],1).repeat(lc_len+1,axis=1)
            dif_al_m=(exp1_2020-exp0_2001).transpose((2,3,0,1))*data_m[m]
            if ismask:
                mask_dif=np.abs((exp1_2020-exp0_2001)/exp0_2001).transpose((2,3,0,1))
                dif_al_m[mask_dif<calibration]=0
            dif_al.append(dif_al_m*mt[m]/365/24/3600)
        dif_al=np.stack(dif_al)
        dif_al=np.nansum(dif_al,axis=-1)
        dif_al=np.nansum(dif_al,axis=-1)
        dif_al=np.nansum(dif_al,axis=0)
        if not mode=='snowfree-change':
            data_m=data_m2
            area_tf=np.nansum(data_m,axis=-1)
            area_tf=np.nansum(area_tf,axis=-1)
            area_tf=np.nanmean(area_tf,axis=0)
            kk.append(kk[0]+np.nansum(dif_al)/np.nansum(area_tf))
        else:
            area_tf=np.nansum(data_m,axis=-1)
            area_tf=np.nansum(area_tf,axis=-1)
            area_tf=np.nanmean(area_tf,axis=0)
            area_tf2=np.nansum(data_m2,axis=-1)
            area_tf2=np.nansum(area_tf2,axis=-1)
            area_tf2=np.nanmean(area_tf2,axis=0)
            snowfree_fraction=area_tf/area_tf2
            kk.append(kk[0]+np.nansum(dif_al)/np.nansum(area_tf))
            dif_al[snowfree_fraction<0.05]=0
        kk_map.append(dif_al/area_tf)
    kk_map=np.stack(kk_map)
    print(kk)
    plt.imshow(kk_map[-1],vmax=0.05,vmin=-0.05,cmap='seismic')
    plt.show()
    return kk,kk_map
    

result=torch.load('../data/albedo_change.pth')
for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
    print(landcover_type)
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15 
    stat=torch.load(f'/data2/hzy/albedo2/data/stat_sw-mean_sza-70_band-shortwave_{landcover_type}.pth')
    _,sza_L=torch.load(root_path+'sza_and_szaL.pth')
    albedo_all=np.zeros((12,20,lc_len+1,180,360)).astype(np.float32)
    area_all=np.zeros((12,20,lc_len+1,180,360)).astype(np.float32)
    mask=np.zeros((12,20,lc_len+1,180,360))

    for y in range(2001,2021):
        for m in range(1,13):
            albedo_all[m-1,y-2001]=stat[f'{y}-{m}-albedo']
            area_all[m-1,y-2001]=stat[f'{y}-{m}-typearea']
            mask[m-1,y-2001]=np.expand_dims(sza_L[y-2001,m-1],0).repeat(lc_len+1,axis=0)
    albedo_all[albedo_all==0]=np.nan
    albedo_all[mask>=1]=np.nan
    area_all=month_fill(area_all,mask>=1)
    albedo_all=month_fill(albedo_all,mask>=1)
    for mode in ['snowfree-change','snowfree','all','snow','conversion','non_conversion']:
        print(mode)
        result[f'{landcover_type}_{mode}_s']=get_albedo_change(landcover_type,mode,False)
    result[f'{landcover_type}_all_c']=get_albedo_change(landcover_type,'all',True)
    result[f'{landcover_type}_snowfree-change_c']=get_albedo_change(landcover_type,'snowfree-change',True)
torch.save(result,'../data/albedo_change.pth')

In [ ]:
#Export the RF map induce different factor change.
def get_RF_map(mode,landcover_type):
    print(landcover_type,mode)
    def filter_mode(data,mode,lc_len):
        if mode=='snowfree':
            data[:,-1,:]=0
            data[:,:,-1]=0
        elif mode=='all':
            pass
        elif mode=='snow':
            data[:,:-1,:-1]=0
        elif mode=='conversion':
            data[:,-1,:]=0
            data[:,:,-1]=0
            for i in range(lc_len+1):
                data[:,i,i]=0       
        elif mode=='non_conversion':
            mask_filter=np.zeros((1,lc_len+1,lc_len+1))
            for i in range(lc_len):
                mask_filter[:,i,i]=1
            data=data*(mask_filter.reshape((1,lc_len+1,lc_len+1,1,1)))
        else:
            raise('ERROR MODE')
        return data
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15 
    kka2=filter_mode(kka.copy(),mode,lc_len)
    kka2=np.sum(kka2,axis=1)
    output=np.sum(kka2,axis=1)/365/24/3600/1e6/area_all
    return output

for landcover_type in ['LCCS1','LCCS2','LCCS3']:
    result={}
    print(landcover_type)
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15 
    mt=[]
    for month in range(1,13):
        now=datetime.datetime.strptime(f'2001-{month}-1', '%Y-%m-%d')
        month_time=(now+relativedelta(months=1)-now).total_seconds()
        mt.append(month_time)
    mt=np.array(mt)
    kka=np.load(f'/data2/hzy/albedo2/data/confusematrix_albedo-multiply-area-multiply-time_{landcover_type}.npy')
    kernels=torch.load(f"/data2/hzy/albedo2/kernel/kernels_6.pth")
    kka=kka*kernels['mean']*1e6
    kka[np.isnan(kka)]=0
    kka=np.sum(kka,axis=3)
    area_all=np.zeros((12,20,lc_len+1,180,360)).astype(np.float32)
    mask=np.zeros((12,20,lc_len+1,180,360))
    stat=torch.load(f'/data2/hzy/albedo2/data/stat_sw-mean_sza-85_band-shortwave_{landcover_type}.pth')
    for y in range(2001,2021):
        for m in range(1,13):
            area_all[m-1,y-2001]=stat[f'{y}-{m}-typearea']
    area_all=month_fill(area_all,mask>=1)
    area_all=np.nanmean(area_all,axis=0)
    area_all=area_all
    area_all=np.nansum(area_all,axis=1)

    if landcover_type=='LCCS1':
        result['snow']=get_RF_map('snow',landcover_type)
        plt.imshow(result['snow'][-1],vmax=10,vmin=-10,cmap='seismic')
        plt.show()
    result['snowfree']=get_RF_map('snowfree',landcover_type)
    plt.imshow(result['snowfree'][-1],vmax=10,vmin=-10,cmap='seismic')
    plt.show()
        
    result['all']=get_RF_map('all',landcover_type)
    plt.imshow(result['all'][-1],vmax=10,vmin=-10,cmap='seismic')
    plt.show()
    
    result['conversion']=get_RF_map('conversion',landcover_type)
    plt.imshow(result['conversion'][-1],vmax=10,vmin=-10,cmap='seismic')
    plt.show()
    
    result['non_conversion']=get_RF_map('non_conversion',landcover_type)
    plt.imshow(result['non_conversion'][-1],vmax=10,vmin=-10,cmap='seismic')
    plt.show()
    torch.save(result,f'../data/map_RF_{landcover_type}.pth')

In [ ]:
#Export the tif files of albedo change map.
all=[]
all_c=[]
snowfree=[]
snowfree_change=[]
snowfree_change_c=[]
snow=[]
result=torch.load('../data/albedo_change.pth')
for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
    for key in ['snow','non_conversion','conversion','all','snowfree','snowfree-change']:
        if key=='snow':
            snow.append(result[f'{landcover_type}_{key}_s'][1][-1])
        elif key in ['non_conversion','conversion']:
            kk=result[f'{landcover_type}_{key}_s'][1][-1]
            kk[np.isnan(kk)]=0
            tif_save(kk,f'../data/albedo_{key}_{landcover_type}_1degree.tif',(-179.5,1,0,89.5,0,-1),p='4326')
        elif key=='all':
            all.append(result[f'{landcover_type}_{key}_s'][1][-1])
            all_c.append(result[f'{landcover_type}_{key}_c'][1][-1])
        elif key=='snowfree':
            snowfree.append(result[f'{landcover_type}_{key}_s'][1][-1])
        elif key=='snowfree-change':
            snowfree_change.append(result[f'{landcover_type}_{key}_s'][1][-1])
            snowfree_change_c.append(result[f'{landcover_type}_{key}_c'][1][-1])
all=np.nanmean(np.stack(all),axis=0)
all[np.isnan(all)]=0
tif_save(all,f'../data/albedo_all_1degree.tif',(-179.5,1,0,89.5,0,-1),p='4326')

all_c=np.nanmean(np.stack(all_c),axis=0)
all_c[np.isnan(all_c)]=0
tif_save(all_c,f'../data/albedo_all_1degree_calibration.tif',(-179.5,1,0,89.5,0,-1),p='4326')

snowfree=np.mean(np.stack(snowfree),axis=0).copy()
snowfree[np.isnan(snowfree)]=0
tif_save(snowfree,f'../data/albedo_snowfree_1degree.tif',(-179.5,1,0,89.5,0,-1),p='4326')

snowfree_change=np.mean(np.stack(snowfree_change),axis=0).copy()
snowfree_change[np.isnan(snowfree_change)]=0
tif_save(snowfree_change,f'../data/albedo_snowfree-change_1degree.tif',(-179.5,1,0,89.5,0,-1),p='4326')

snowfree_change_c=np.mean(np.stack(snowfree_change_c),axis=0).copy()
snowfree_change_c[np.isnan(snowfree_change_c)]=0
tif_save(snowfree_change_c,f'../data/albedo_snowfree-change_1degree_calibration.tif',(-179.5,1,0,89.5,0,-1),p='4326')

snow=np.mean(np.stack(snow),axis=0).copy()
snow[np.isnan(snow)]=0
tif_save(snow,f'../data/albedo_snow_1degree_calibration.tif',(-179.5,1,0,89.5,0,-1),p='4326')


In [10]:
#Export the tif files of RF map.
snow=[]
all=[]
snowfree=[]
for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
    result=torch.load(f'../data/map_RF_{landcover_type}.pth')
    for key in result.keys():
        if key=='snow':
            snow.append(result[key][-1]-result[key][0])
        elif key in ['non_conversion','conversion']:
            kk=result[key][-1]-result[key][0]
            kk[np.isnan(kk)]=0
            tif_save(kk,f'../data/RF_{key}-{landcover_type}_1degree.tif',(-180,1,0,90,0,-1),p='4326')
        elif key=='all':
            all.append(result[key][-1]-result[key][0])
        elif key=='snowfree':
            snowfree.append(result[key][-1]-result[key][0])
all=np.nanmean(np.stack(all),axis=0)
all[np.isnan(all)]=0
tif_save(all,f'../data/RF_all_1degree.tif',(-180,1,0,90,0,-1),p='4326')

snowfree=np.nanmean(np.stack(snowfree),axis=0)
snowfree[np.isnan(snowfree)]=0
tif_save(snowfree,f'../data/RF_snowfree_1degree.tif',(-180,1,0,90,0,-1),p='4326')

snow=np.nanmean(np.stack(snow),axis=0)
snow[np.isnan(snow)]=0
tif_save(snowfree,f'../data/RF_snow_1degree.tif',(-180,1,0,90,0,-1),p='4326')

In [ ]:
#Export contribution map of PV, NPV and SWC basde on LCCS2
landcover_type='LCCS2'
result={}
lc_len=10
k=[]
for i in range(40076):
    k.append((pi/180.0)*R*R*abs(math.sin((90.00220831593487-pixel_with*i)/180.0*pi) - math.sin((90.00220831593487-pixel_with*(i+1))/180.0*pi)) * pixel_with)
lc01=rasterio.open(f'/data2/hzy/albedo2/LCCS/LCCS/2001_{landcover_type}_v2.tif').read(1)
lc20=rasterio.open(f'/data2/hzy/albedo2/LCCS/LCCS/2020_{landcover_type}_v2.tif').read(1)
lc01[lc01==100]=20
lc01[lc01==0]=20
lc20[lc20==100]=20
lc20[lc20==0]=20

z=np.zeros(lc01.shape)
z[lc01==lc20]=1
for i in [1,2,20]:
    z[lc01==i]=0
lc01[lc01==20]=0
lc01[lc20==20]=0
lc01[lc01!=0]=1
landarea=np.array(k).reshape(-1,1)*lc01

for b in ['NDVI','SSI','NMDI']:
    for key in ['mean']:
        print(b)
        img=rasterio.open(f'/data2/hzy/ssd_hzy/G3/{b}_{key}_eg_final.tif').read(1)
        img[np.isnan(img)]=0
        img=z*img
        tif_save(img,f'/data2/hzy/ssd_hzy/G3/{b}_{key}_eg_final_map.tif',global_trf,p='4326')
for b in ['NDVI','SSI','NMDI']:
    print(b)
    img=rasterio.open(f'/data2/hzy/ssd_hzy/G3/{b}_final.tif').read(1)
    img[np.isnan(img)]=0
    img=z*img
    tif_save(img,f'/data2/hzy/ssd_hzy/G3/{b}_final_map.tif',global_trf,p='4326')
 


In [ ]:
# Monthly contribution of PV, NPV and SWC to GLMA
for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
    k=[]
    for i in range(40076):
        k.append((pi/180.0)*R*R*abs(math.sin((90.00220831593487-pixel_with*i)/180.0*pi) - math.sin((90.00220831593487-pixel_with*(i+1))/180.0*pi)) * pixel_with)
    lc01=rasterio.open(f'/data2/hzy/albedo2/LCCS/LCCS/2001_{landcover_type}_v2.tif').read(1)
    lc20=rasterio.open(f'/data2/hzy/albedo2/LCCS/LCCS/2020_{landcover_type}_v2.tif').read(1)
    lc01[lc01==100]=20
    lc01[lc01==0]=20
    lc20[lc20==100]=20
    lc20[lc20==0]=20
    z=np.zeros(lc01.shape)
    z[lc01==lc20]=1
    if landcover_type=='IGBP':
        for i in [2,16,20]:
            z[lc01==i]=0
    else:
        for i in [1,2,20]:
            z[lc01==i]=0
    lc01[lc01==20]=0
    lc01[lc20==20]=0
    lc01[lc01!=0]=1
    landarea=np.array(k).reshape(-1,1)*lc01
    del lc01,lc20,k
    gc.collect()
    print(landarea.sum())
    ddd={}
    for b in ['NDVI','SSI','NMDI']:
        print(b)
        ddd[b]=[]
        for m in tqdm(range(1,13)):
            img=rasterio.open(f'/data2/hzy/ssd_hzy/G3/{b}{m}_albedo_final.tif').read(1)
            img[np.isnan(img)]=0
            img=img*z
            img=landarea*img
            ddd[b].append(img.sum()/landarea.sum())
            del img
            gc.collect()
    torch.save(ddd,f'seasons_model_dict_{landcover_type}.pth')

In [ ]:
# Contribution of PV, NPV and SWC to GLMA over different LULC types
for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
    result={}
    print(landcover_type)
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15 
    k=[]
    for i in range(40076):
        k.append((pi/180.0)*R*R*abs(math.sin((90.00220831593487-pixel_with*i)/180.0*pi) - math.sin((90.00220831593487-pixel_with*(i+1))/180.0*pi)) * pixel_with)
        
    lc01=rasterio.open(f'/data2/hzy/albedo2/LCCS/LCCS/2001_{landcover_type}_v2.tif').read(1)
    lc20=rasterio.open(f'/data2/hzy/albedo2/LCCS/LCCS/2020_{landcover_type}_v2.tif').read(1)
    lc01[lc01==100]=20
    lc01[lc01==0]=20
    lc20[lc20==100]=20
    lc20[lc20==0]=20

    z=np.zeros(lc01.shape)
    z[lc01==lc20]=lc01[lc01==lc20]
    if landcover_type=='IGBP':
        for i in [2,16,20]:
            z[lc01==i]=0
    else:
        for i in [1,2,20]:
            z[lc01==i]=0
    lc01[lc01==20]=0
    lc01[lc20==20]=0
    lc01[lc01!=0]=1



    landarea=np.array(k).reshape(-1,1)*lc01
    del lc01,lc20,k
    gc.collect()
    print(landarea.sum())

    
    for b in ['NDVI','SSI','NMDI']:
        result_b=[]
        print(b)
        img=rasterio.open(f'/data2/hzy/ssd_hzy/G3/{b}_final.tif').read(1)
        img[np.isnan(img)]=0
        img[z==0]=0
        img=landarea*img
        for i in range(1,lc_len+1):
            img2=img.copy()
            img2[z!=i]=0
            zz=img2.sum()/landarea.sum()
            result_b.append(zz)
            print(type_code[landcover_type][i-1],zz)
            
            del img2
            gc.collect()
        result[b]=result_b
        del img
        gc.collect()
    
    del z
    gc.collect()
    torch.save(result,f'type_model_dict_{landcover_type}.pth')

In [ ]:
# functions to calculte trend and significance test 
def get_ALLUMs_mean():
    _,sza_L=torch.load(root_path+'sza_and_szaL.pth')
    albedo_all_four=[]
    area_all_four=[]
    for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
        print(landcover_type)
        if landcover_type=='LCCS2':
            lc_len=10
        elif landcover_type=='LCCS3':
            lc_len=9
        elif landcover_type=='IGBP':
            lc_len=16
        elif landcover_type=='LCCS1':
            lc_len=15 
        
        stat=torch.load(f'/data2/hzy/albedo2/data/stat_sw-mean_sza-70_band-shortwave_{landcover_type}.pth')
        albedo_all=np.zeros((12,20,lc_len+1,180,360)).astype(np.float32)
        area_all=np.zeros((12,20,lc_len+1,180,360)).astype(np.float32)
        mask=np.zeros((12,20,lc_len+1,180,360))

        for y in range(2001,2021):
            for m in range(1,13):
                albedo_all[m-1,y-2001]=stat[f'{y}-{m}-albedo']
                area_all[m-1,y-2001]=stat[f'{y}-{m}-typearea']
                mask[m-1,y-2001]=np.expand_dims(sza_L[y-2001,m-1],0).repeat(lc_len+1,axis=0)
        albedo_all[albedo_all==0]=np.nan
        albedo_all[mask>=1]=np.nan
        area_all=month_fill(area_all,mask>=1)
        albedo_all=month_fill(albedo_all,mask>=1)
        zz=area_all*albedo_all
        area_snowfree=np.nansum(area_all[:,:,:-1],axis=2)
        area_snow=area_all[:,:,-1]
        albedo_snowfree=np.nansum(zz[:,:,:-1],axis=2)/area_snowfree
        albedo_snow=zz[:,:,-1]/area_snow
        albedo_two=np.stack([albedo_snowfree,albedo_snow]).transpose([1,2,0,3,4])
        area_two=np.stack([area_snowfree,area_snow]).transpose([1,2,0,3,4])
        albedo_all_four.append(albedo_two)
        area_all_four.append(area_two)
    albedo_all=np.nanmean(np.stack(albedo_all_four),axis=0)
    area_all=np.nanmean(np.stack(area_all_four),axis=0)
    return albedo_all,area_all

def trend_map(a):
    def LinearRegression_trend(kk):
        if np.isnan(kk).any():
            mask=0
            return [-1,0,mask]
        else:
            mask=1
            result = mk(kk)
            return [result[-1],result[0],mask]
    k=[LinearRegression_trend(a[i]) for i in range(a.shape[0])]
    ps=[k[i][0] for i in range(len(k))]
    slopes=[k[i][1] for i in range(len(k))]
    masks=[k[i][2] for i in range(len(k))]
    return [ps,slopes,masks]

def get_land_perc():
    land_area=np.nanmean(area_all,axis=0)
    land_area=np.nanmean(land_area,axis=0)
    land_area=np.nansum(land_area,axis=0)
    pi = 3.1415926
    R = 6371007.181	
    pixel_with=1
    lats=[90-i*1 for i in range(180)]
    pixels_area=[(pi/180.0)*R*R*abs(math.sin((lat)/180.0*pi) - math.sin((lat-pixel_with)/180.0*pi)) * pixel_with for lat in lats]
    area_map=np.array([[1]*360])*(np.array([pixels_area]).T)/1000000
    land_area=land_area.reshape((60,-1,360)).reshape((60,3,120,-1)).transpose((0,2,1,3))
    land_area=np.sum(land_area,axis=-1)
    land_area=np.sum(land_area,axis=-1)
    area_map=area_map.reshape((60,-1,360)).reshape((60,3,120,-1)).transpose((0,2,1,3))
    area_map=np.sum(area_map,axis=-1)
    area_map=np.sum(area_map,axis=-1)
    land_perc=land_area/area_map
    return land_perc

def get_p(zz,nan=True):
    zz2=zz.reshape((20,60,-1,360)).reshape((20,60,3,120,-1)).transpose((0,1,3,2,4)).reshape((20,60,120,-1))
    if nan:
        zz2=np.nanmean(zz2,axis=-1).reshape(20,-1).transpose((1,0))
    else:
        zz2=np.mean(zz2,axis=-1).reshape(20,-1).transpose((1,0))
    rr=trend_map(zz2)
    slope_3=np.array(rr[1]).reshape(60,120)
    p=np.array(rr[0]).reshape(60,120)
    p[p==-1]=1
    p[p<0.05]=0
    p[p>=0.05]=1
    p[p==0]=2
    p-=1
    return p,slope_3

def get_calibration_mask(input_albedo,slope,mask=None):
    calibration_limit=input_albedo[0]*0.0007
    calibration_mask=np.zeros((180,360))
    calibration_mask[slope>calibration_limit]=1
    calibration_mask[slope<(-calibration_limit)]=1
    if mask is not None:
        calibration_mask[mask==0]=1
    calibration_mask[slope==0]=1
    calibration_limit_nan=np.zeros((180, 360))
    calibration_limit_nan[slope>calibration_limit]=1
    calibration_limit_nan[slope<(-calibration_limit)]=1
    calibration_limit_nan[calibration_limit_nan==0]=np.nan
    return calibration_mask,calibration_limit_nan
def to_gpd(p):
    lat=np.array([88.5-3*i for i in range(60)]).reshape((60,1))*np.ones((1,120))
    lon=np.array([-178.5+3*i for i in range(120)]).reshape((1,120))*np.ones((60,1))
    lat[p==0]=0
    lon[p==0]=0
    lat=list(lat.reshape((-1,1)))
    lon=list(lon.reshape((-1,1)))
    coords=[[lat[i],lon[i]] for i in range(len(lat)) if lat[i]!=0]
    from shapely import geometry
    coords_geo=[geometry.Point(i[1],i[0]) for i in coords]
    gpdd=gpd.GeoDataFrame.from_dict({'id':list(range(len(coords_geo))),'geometry':coords_geo})
    gpdd=gpdd.set_crs(4326)
    return gpdd
def calculation_sig_perc(area,area_mask,p,slope):
    # area_mask=area*calibration_limit_nan
    area2=area.reshape((20,60,-1,360)).reshape((20,60,3,120,-1)).transpose((0,1,3,2,4)).reshape((20,60,120,-1))
    area2_mask=area_mask.reshape((20,60,-1,360)).reshape((20,60,3,120,-1)).transpose((0,1,3,2,4)).reshape((20,60,120,-1))
    area2=np.nansum(area2,axis=-1)
    area2=np.nanmean(area2,axis=0)
    area2_mask=np.nansum(area2_mask,axis=-1)
    area2_mask=np.nanmean(area2_mask,axis=0)
    area2_mask[np.isnan(area2_mask)]=0
    area_sig=area2_mask.copy()
    area_sig[p==0]=0
    area_increase=area_sig.copy()
    area_increase[slope<0]=0
    area_decrease=area_sig.copy()
    area_decrease[slope>0]=0
    print(area_sig.sum()/area2.sum()*100,'%',area_increase.sum()/area2.sum()*100,'%',area_decrease.sum()/area2.sum()*100,'%')

def t_test(a,b):
    if len(a.shape)==1:
        a=a.reshape((1,-1))
    if len(b.shape)==1:
        b=b.reshape((1,-1))
    def t_test_single(a_single,b_single):
        r=stats.ttest_ind(b_single, a_single)
        return [(r.__getattribute__("statistic")>0).astype(np.int8),(r.__getattribute__("pvalue")<0.05).astype(np.int8)]
    k=[t_test_single(a[i],b[i]) for i in range(a.shape[0])]
    k2=[k[i][0] for i in range(len(k))]
    k3=[k[i][1] for i in range(len(k))]
    return [k2,k3]

def get_p_twosided(zz,nan=True):
    zz_a2=np.stack(zz[:10]).reshape((10,60,-1,360)).reshape((10,60,3,120,-1)).transpose((0,1,3,2,4)).reshape((10,60,120,-1))
    zz_b2=np.stack(zz[10:]).reshape((10,60,-1,360)).reshape((10,60,3,120,-1)).transpose((0,1,3,2,4)).reshape((10,60,120,-1))
    if nan:
        zz_a2=np.nanmean(zz_a2,axis=-1).reshape(10,-1).transpose((1,0))
        zz_b2=np.nanmean(zz_b2,axis=-1).reshape(10,-1).transpose((1,0))
    else:
        zz_a2=np.mean(zz_a2,axis=-1).reshape(10,-1).transpose((1,0))
        zz_b2=np.mean(zz_b2,axis=-1).reshape(10,-1).transpose((1,0))
    t2,p2=t_test(zz_a2,zz_b2)
    p_array2=np.array(p2).reshape(60,120)
    t_array2=np.array(t2).reshape(60,120)
    t_array2[t_array2==0]=-1
    return t_array2,p_array2

def get_area_mask(mode):
    _,sza_L=torch.load(root_path+'sza_and_szaL.pth')
    area_mask=[]
    for landcover_type in tqdm(['LCCS1','LCCS2','LCCS3','IGBP']):
        if landcover_type=='LCCS2':
            lc_len=10
        elif landcover_type=='LCCS3':
            lc_len=9
        elif landcover_type=='IGBP':
            lc_len=16
        elif landcover_type=='LCCS1':
            lc_len=15 
        
        stat=torch.load(f'/data2/hzy/albedo2/data/stat_sw-mean_sza-70_band-shortwave_{landcover_type}.pth')
        _,sza_L=torch.load(root_path+'sza_and_szaL.pth')
        albedo_all=np.zeros((12,20,lc_len+1,180,360)).astype(np.float32)
        area_all=np.zeros((12,20,lc_len+1,180,360)).astype(np.float32)
        mask=np.zeros((12,20,lc_len+1,180,360))

        for y in range(2001,2021):
            for m in range(1,13):
                albedo_all[m-1,y-2001]=stat[f'{y}-{m}-albedo']
                area_all[m-1,y-2001]=stat[f'{y}-{m}-typearea']
                mask[m-1,y-2001]=np.expand_dims(sza_L[y-2001,m-1],0).repeat(lc_len+1,axis=0)
        albedo_all[albedo_all==0]=np.nan
        albedo_all[mask>=1]=np.nan
        area_all=month_fill(area_all,mask>=1)
        albedo_all=month_fill(albedo_all,mask>=1)
        
        data_m=torch.load(f"/data2/hzy/albedo2/confuse_matrix/confuse_matrix_{y}_{landcover_type}.pth")
        mask_t=np.expand_dims(mask[:,0,:],0).repeat(lc_len+1,axis=0).transpose((1,3,4,0,2))
        data_m[mask_t>=1]=np.nan
        data_m=month_fill(data_m,mask_t>=1)
        data_m=filter_mode(data_m,mode,lc_len)
        calibration=(y-2001)*0.0007
        for m in range(12):
            exp1_2020=np.expand_dims(albedo_all[m,y-2001],0).repeat(lc_len+1,axis=0)
            exp0_2001=np.expand_dims(albedo_all[m,0],1).repeat(lc_len+1,axis=1)
            mask_dif=np.abs((exp1_2020-exp0_2001)/exp0_2001).transpose((2,3,0,1))
            data_m[m][mask_dif<calibration]=0
        data_m=data_m*(mt.reshape(12,1,1,1,1)/365/24/3600)
        area_tf=np.nansum(data_m,axis=-1)
        area_tf=np.nansum(area_tf,axis=-1)
        area_tf=np.nansum(area_tf,axis=0)
        area_mask.append(area_tf)
    area_mask2=np.nanmean(np.stack(area_mask),axis=0)
    return area_mask2

In [ ]:
#Trend for albedo and MK test
mt=[]
for month in range(1,13):
    now=datetime.datetime.strptime(f'2001-{month}-1', '%Y-%m-%d')
    month_time=(now+relativedelta(months=1)-now).total_seconds()
    mt.append(month_time)
mt=np.array(mt)
albedo_all,area_all=get_ALLUMs_mean()
area_all=area_all*(mt.reshape(12,1,1,1,1)/365/24/3600)
land_perc=get_land_perc()
torch.save(land_perc,'../data/land_perc.pth')
for key in ['all','snowfree','snow-change5perc']:
    print(key)
    zz=albedo_all*area_all
    if key=='snow-change5perc':
        zz=np.nansum(zz[:,:,-1],axis=0)
        # zz[zz==0]=np.nan
        area=np.nansum(area_all[:,:,-1],axis=0)
    elif key=='snowfree':
        zz=np.nansum(np.nansum(zz[:,:,:-1],axis=0),axis=1)
        area=np.nansum(np.nansum(area_all[:,:,:-1],axis=0),axis=1)
    elif key=='all':
        zz=np.nansum(np.nansum(zz,axis=0),axis=1)
        area=np.nansum(np.nansum(area_all,axis=0),axis=1)
    else:
        raise()
    
    all_albedo=zz/area
    zz=all_albedo.copy()

    if not key=='all':
        if key=='snow-change5perc':
            mean_area=np.nansum(np.nansum(area_all,axis=0),axis=0)
            snow_area_perc=mean_area[-1]/np.nansum(mean_area,axis=0)
        else:
            mean_area=np.nansum(np.nansum(area_all,axis=0),axis=0)
            snow_area_perc=mean_area[0]/np.nansum(mean_area,axis=0)

        mask=(snow_area_perc>0.05).astype(np.float32)
        mask[mask==0]=np.nan
        zz=zz*mask
        area*=mask
        area[np.isnan(area)]=0
    else:
        mask=None

    rr=trend_map(zz.reshape((20,-1)).transpose((1,0)))
    slope=np.array(rr[1]).reshape(180,360)
    tif_save(slope,f'../data/albedo_{key}_trend_1degree.tif',(-180, 1.0, 0.0, 90, 0.0, -1.0),p='4326')
    calibration_mask,calibration_limit_nan=get_calibration_mask(all_albedo,slope,mask)
    tif_save(calibration_mask,f'../data/albedo_{key}_trend_1degree_mask.tif',(-180, 1.0, 0.0, 90, 0.0, -1.0),p='4326')

    zz_calibration_limit=zz*calibration_limit_nan
    p,slope_3=get_p(zz_calibration_limit,nan=True)
    p2=p.copy()
    p2[land_perc<0.05]=0
    gpdd=to_gpd(p2)
    gpdd.to_file(f'../data/significant_{key}_mannkendall_calibrationmask.shp')
    calculation_sig_perc(area,area*calibration_limit_nan,p,slope_3)
    p,slope_3=get_p(zz,nan=True)
    p2=p.copy()
    p2[land_perc<0.05]=0
    gpdd=to_gpd(p2)
    gpdd.to_file(f'../data/significant_{key}_mannkendall.shp')
    calculation_sig_perc(area,area,p,slope_3)


In [ ]:
#Trend for RF and MK test
for key in ['all']:
    print(key)
    kk=[]
    for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
        result=torch.load(f'../data/map_RF_{landcover_type}.pth')
        kk.append(result['all'])
    zz=np.nanmean(np.stack(kk),axis=0)
    area=np.nansum(np.nansum(area_all,axis=0),axis=1)
    mask=None
    rr=trend_map(zz.reshape((20,-1)).transpose((1,0)))
    slope=np.array(rr[1]).reshape(180,360)
    tif_save(slope,f'../data/RF_{key}_trend_1degree.tif',(-180, 1.0, 0.0, 90, 0.0, -1.0),p='4326')
 
    p,slope_3=get_p(zz,nan=True)
    p2=p.copy()
    p2[land_perc<0.05]=0
    gpdd=to_gpd(p2)
    gpdd.to_file(f'../data/RF_significant_{key}_mannkendall.shp')
    calculation_sig_perc(area,area,p,slope_3)

In [ ]:
#Two-sided t test for albedo
area_mask_all=get_area_mask('all')
area_mask_snowfree=get_area_mask('snowfree-change')
result=torch.load('../data/albedo_change.pth')
land_perc=torch.load('../data/land_perc.pth')
for key in ['all','snowfree-change']:
    if key=='all':
        area=np.nansum(np.nansum(area_all[:,:,:],axis=0),axis=1)
    elif key=='snowfree-change':
        area=np.nansum(np.nansum(area_all[:,:,:-1],axis=0),axis=1)
    else:
        raise()
    for mode in ['s','c']:
        kk=[]
        for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
            kk.append(result[f'{landcover_type}_{key}_{mode}'][1])
        zz=np.nanmean(np.stack(kk),axis=0)
        t,p=get_p_twosided(zz,nan=True)
        p2=p.copy()
        p2[land_perc<0.05]=0
        gpdd=to_gpd(p2)
        gpdd.to_file(f'../data/significant_{key}_{mode}.shp')
        print(key,mode)
        if mode=='c':
            if key=='all':
                calculation_sig_perc(area,np.expand_dims(area_mask_all, 0).repeat(20, axis=0),p,t)
            else:
                calculation_sig_perc(area,np.expand_dims(area_mask_snowfree, 0).repeat(20, axis=0),p,t)
        else:
            calculation_sig_perc(area,area,p,t)

In [ ]:
#Two-sided t test for RF
for key in ['all']:
    area=np.nansum(np.nansum(area_all[:,:,:],axis=0),axis=1)
    kk=[]
    for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
        result=torch.load(f'../data/map_RF_{landcover_type}.pth')
        kk.append(result['all'])
    zz=np.nanmean(np.stack(kk),axis=0)
    
    t,p=get_p_twosided(zz,nan=True)
    p2=p.copy()
    p2[land_perc<0.05]=0
    gpdd=to_gpd(p2)
    gpdd.to_file(f'../data/RF_significant_{key}.shp')
    print(key,mode)
    calculation_sig_perc(area,area,p,t)